In [6]:
import argparse
import copy
import json
import os
import sys

os.chdir("/home/nakamura/inflated_convnets_pytorch/")

import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from IPython import embed
from matplotlib import pyplot as plt

from src.i3dense import I3DenseNet
from src.i3res import I3ResNet

%matplotlib inline

In [7]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
dataset = datasets.ImageFolder('data/dummy-dataset',
                               transforms.Compose([
                                   transforms.CenterCrop(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   normalize,
                               ]))

class_idx = json.load(open('data/imagenet_class_index.json'))
imagenet_classes = [class_idx[str(k)][1] for k in range(len(class_idx))]



In [8]:
resnet = torchvision.models.resnet50(pretrained=True)
densenet = torchvision.models.densenet169(pretrained=True)

/home/nakamura/.conda/envs/video/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [7]:
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [8]:
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [10]:
i3resnet = I3ResNet(copy.deepcopy(resnet), 16)
i3densenet = I3DenseNet(copy.deepcopy(densenet), 16, inflate_block_convs=True)

In [12]:
i3densenet.features.conv0.weight.shape

torch.Size([64, 3, 1, 7, 7])

In [13]:
for i, (input_2d, target) in enumerate(loader):
    
        target_var = torch.autograd.Variable(target)
        input_2d_var = torch.autograd.Variable(input_2d)

In [32]:
resnet.eval()
i3resnet.eval()
#densenet.eval()
# i3densenet.eval()

I3ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck3d(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
    

torch.nn.modules.conv.Conv3d

In [25]:
o2d = resnet(input_2d_var)
# o2d = densenet(input_2d_var)

In [26]:
input_3d = input_2d.unsqueeze(2).repeat(1, 1, 16, 1, 1)
input_3d_var = torch.autograd.Variable(input_3d)

out3d = i3resnet(input_3d_var)
# out3d = i3densenet(input_3d_var)

In [17]:
re = i3densenet.features.conv0.weight

In [18]:
re[0][2][0]

tensor([[-0.0417,  0.0374, -0.0592,  0.0340, -0.0718,  0.0629, -0.0469],
        [ 0.0733, -0.0756,  0.0889, -0.0476,  0.1333, -0.1627,  0.1147],
        [-0.0467,  0.0538,  0.0289, -0.1911,  0.0510,  0.0981, -0.1086],
        [ 0.0494, -0.0352, -0.1733,  0.5272, -0.4474,  0.1504,  0.0110],
        [-0.0825,  0.1483,  0.0194, -0.4137,  0.4615, -0.2388,  0.0455],
        [ 0.0598, -0.1716,  0.1396,  0.1027, -0.2098,  0.1354, -0.0345],
        [-0.0284,  0.0956, -0.1265,  0.0404,  0.0283, -0.0240,  0.0101]],
       grad_fn=<SelectBackward>)

In [19]:
re[1]

tensor([[[[-0.0124, -0.0042,  0.0634,  0.0147, -0.0582, -0.0483, -0.0051],
          [-0.0055, -0.0465,  0.1243,  0.1283, -0.0801, -0.0434,  0.0037],
          [-0.0014, -0.1201,  0.0349,  0.2919, -0.0808, -0.1060, -0.0022],
          [ 0.0125, -0.1254, -0.1034,  0.3776,  0.0926, -0.1554, -0.0236],
          [ 0.0191, -0.0585, -0.1907,  0.1829,  0.2773, -0.1029, -0.0619],
          [ 0.0091, -0.0144, -0.1578, -0.0070,  0.2052,  0.0434, -0.0548],
          [-0.0044,  0.0078, -0.1058, -0.0658,  0.0812,  0.0591,  0.0057]]],


        [[[-0.0238, -0.0351,  0.0744,  0.0406, -0.0601, -0.0623, -0.0109],
          [-0.0203, -0.0964,  0.1427,  0.1788, -0.0907, -0.0620, -0.0031],
          [-0.0192, -0.1834,  0.0365,  0.3881, -0.0735, -0.1461, -0.0198],
          [ 0.0032, -0.1855, -0.1681,  0.4692,  0.1664, -0.1746, -0.0494],
          [ 0.0107, -0.1147, -0.2910,  0.2082,  0.3723, -0.0768, -0.0721],
          [ 0.0393, -0.0254, -0.2122, -0.0348,  0.2498,  0.0775, -0.0613],
          [ 0.0086,  

In [27]:
o2d

tensor([[-3.1352,  0.5686, -0.3020, -0.8421, -2.4755,  0.4750, -2.5059,  0.4557,
          1.6123, -2.4777, -2.1154, -2.9493, -3.9388, -2.3731, -3.0658, -2.8181,
         -2.6578,  1.1197, -1.4043, -4.7591, -0.9188, -2.7249, -2.0153,  0.6596,
         -3.9580, -0.9696, -4.0192, -2.4665, -1.2398, -2.9984, -0.9034, -2.6463,
         -2.8501,  0.7978,  1.6862, -0.0435, -0.2760, -0.5255, -1.8023, -0.2209,
         -1.3988, -2.3441, -0.7008, -1.1933, -1.6738,  0.5851, -0.9874, -1.5761,
         -1.4122, -0.0936, -1.0265,  1.1201, -1.2572, -0.5665,  0.6027,  0.6223,
         -1.7992, -2.2367, -1.6462,  0.1308,  1.0655,  0.4588,  0.8249, -0.0340,
          0.7624, -0.5482,  0.2067,  1.4092, -0.9917, -1.8442, -3.1534,  0.5715,
         -5.3509, -4.2268, -2.3494, -4.9101, -1.3518, -1.3526, -2.1727, -1.1525,
         -0.7463, -3.5708, -0.7887, -1.8128, -1.6380, -3.8108, -1.0554, -3.6473,
          0.7743, -1.0726, -0.9451, -2.7253,  0.5058, -0.8943, -0.8228, -4.9976,
         -2.8386, -0.0213, -

In [28]:
out3d

tensor([[-3.1352,  0.5686, -0.3020, -0.8421, -2.4755,  0.4750, -2.5059,  0.4557,
          1.6123, -2.4777, -2.1154, -2.9493, -3.9388, -2.3731, -3.0658, -2.8181,
         -2.6578,  1.1197, -1.4043, -4.7591, -0.9188, -2.7249, -2.0153,  0.6596,
         -3.9580, -0.9696, -4.0192, -2.4665, -1.2398, -2.9984, -0.9034, -2.6463,
         -2.8501,  0.7978,  1.6862, -0.0435, -0.2760, -0.5255, -1.8023, -0.2209,
         -1.3988, -2.3441, -0.7008, -1.1933, -1.6738,  0.5851, -0.9874, -1.5761,
         -1.4122, -0.0936, -1.0265,  1.1201, -1.2572, -0.5665,  0.6027,  0.6223,
         -1.7992, -2.2367, -1.6462,  0.1308,  1.0655,  0.4587,  0.8249, -0.0340,
          0.7624, -0.5482,  0.2067,  1.4092, -0.9917, -1.8442, -3.1534,  0.5715,
         -5.3509, -4.2268, -2.3494, -4.9101, -1.3518, -1.3526, -2.1727, -1.1525,
         -0.7463, -3.5708, -0.7887, -1.8128, -1.6380, -3.8108, -1.0554, -3.6473,
          0.7743, -1.0726, -0.9451, -2.7253,  0.5058, -0.8943, -0.8228, -4.9976,
         -2.8386, -0.0213, -

In [30]:
for parameter in i3resnet.parameters():
    print(parameter.shape)



torch.Size([64, 3, 3, 7, 7])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 1, 1, 1])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3, 3])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([256, 64, 1, 1, 1])
torch.Size([0])
torch.Size([256])
torch.Size([256])
torch.Size([256, 64, 1, 1, 1])
torch.Size([0])
torch.Size([256])
torch.Size([256])
torch.Size([64, 256, 1, 1, 1])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3, 3])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([256, 64, 1, 1, 1])
torch.Size([0])
torch.Size([256])
torch.Size([256])
torch.Size([64, 256, 1, 1, 1])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3, 3])
torch.Size([0])
torch.Size([64])
torch.Size([64])
torch.Size([256, 64, 1, 1, 1])
torch.Size([0])
torch.Size([256])
torch.Size([256])
torch.Size([128, 256, 1, 1, 1])
torch.Size([0])
torch.Size([128])
torch.Size([128])
torch.Size([128, 128, 3

In [18]:
time_dim=3
time_padding=0
time_stride=1
time_dilation=1
center=False


kernel_dim = (time_dim, conv2d.kernel_size[0], conv2d.kernel_size[1])
padding = (time_padding, conv2d.padding[0], conv2d.padding[1])
stride = (time_stride, conv2d.stride[0], conv2d.stride[0])
dilation = (time_dilation, conv2d.dilation[0], conv2d.dilation[1])
conv3d = torch.nn.Conv3d(
    conv2d.in_channels,
    conv2d.out_channels,
    kernel_dim,
    padding=padding,
    dilation=dilation,
    stride=stride)
# Repeat filter time_dim times along time dimension
weight_2d = conv2d.weight.data
if center:
    weight_3d = torch.zeros(*weight_2d.shape)
    weight_3d = weight_3d.unsqueeze(2).repeat(1, 1, time_dim, 1, 1)
    middle_idx = time_dim // 2
    weight_3d[:, :, middle_idx, :, :] = weight_2d
else:
    weight_3d = weight_2d.unsqueeze(2).repeat(1, 1, time_dim, 1, 1)
    weight_3d = weight_3d / time_dim
    

In [22]:
class Parameter(torch.Tensor):
    r"""A kind of Tensor that is to be considered a module parameter.

    Parameters are :class:`~torch.Tensor` subclasses, that have a
    very special property when used with :class:`Module` s - when they're
    assigned as Module attributes they are automatically added to the list of
    its parameters, and will appear e.g. in :meth:`~Module.parameters` iterator.
    Assigning a Tensor doesn't have such effect. This is because one might
    want to cache some temporary state, like last hidden state of the RNN, in
    the model. If there was no such class as :class:`Parameter`, these
    temporaries would get registered too.

    Arguments:
        data (Tensor): parameter tensor.
        requires_grad (bool, optional): if the parameter requires gradient. See
            :ref:`excluding-subgraphs` for more details. Default: `True`
    """
    def __new__(cls, data=None, requires_grad=True):
        if data is None:
            data = torch.Tensor()
        return torch.Tensor._make_subclass(cls, data, requires_grad)

    def __repr__(self):
        return 'Parameter containing:\n' + super(Parameter, self).__repr__()

    def __reduce_ex__(self, proto):
        return Parameter, (super(Parameter, self), self.requires_grad)
Parameter(weight_3d)

Parameter containing:
tensor([[[[[ 4.4449e-03,  4.8879e-03, -5.1170e-03,  ...,
            -1.3632e-02, -1.4345e-02, -2.3585e-02],
           [ 1.3735e-03,  1.9492e-03,  4.9826e-03,  ...,
             7.3535e-04, -6.9708e-03, -1.2839e-02],
           [ 7.4436e-03,  7.8649e-03,  5.3734e-03,  ...,
             3.4270e-02,  2.0880e-02,  1.7326e-02],
           ...,
           [-3.0116e-04,  9.2558e-03, -3.3684e-03,  ...,
            -4.2406e-02, -2.5535e-02,  2.6151e-03],
           [ 1.1965e-03,  1.6002e-02,  2.0684e-02,  ...,
             8.0890e-03, -1.1221e-02, -5.2363e-03],
           [-2.6676e-02, -1.0746e-02, -5.9359e-03,  ...,
             1.1786e-02,  7.4796e-03,  5.6923e-04]],

          [[ 4.4449e-03,  4.8879e-03, -5.1170e-03,  ...,
            -1.3632e-02, -1.4345e-02, -2.3585e-02],
           [ 1.3735e-03,  1.9492e-03,  4.9826e-03,  ...,
             7.3535e-04, -6.9708e-03, -1.2839e-02],
           [ 7.4436e-03,  7.8649e-03,  5.3734e-03,  ...,
             3.4270e-02,  2.088

In [29]:
conv3d.weight = torch.nn.Parameter(weight_3d)

In [28]:
type(torch.nn.Parameter(weight_3d))

torch.nn.parameter.Parameter